# Issue Commands to Change Settings

In [1]:
import os
import numpy as np

import context

from gopro_helper import api
from gopro_helper import json_io


In [3]:
# Load JSON API information
api_details = json_io.read(api.fname_json)

In [12]:
# Example status group details


'/gp/gpMediaList'

In [4]:
# Example status group details
api_details['status']['groups'][1]

{'fields': [{'id': 43, 'name': 'mode'},
  {'id': 44, 'name': 'sub_mode'},
  {'id': 71, 'name': 'video_selected_flatmode'},
  {'id': 72, 'name': 'photo_selected_flatmode'},
  {'id': 73, 'name': 'timelapse_selected_flatmode'}],
 'group': 'app'}

# Camera Current Settings

In [7]:
api.get_settings()

{'Setup': {'Auto Off': '30 MIN', 'Current Flat Mode': 'Video', 'GPS': 'ON'},
 'System': {'Current Time Msec': 1170312,
  'Internal Battery Percentage': 83,
  'Num Total Photos': 8448,
  'Num Total Videos': 6,
  'Remaining Photos': 6317,
  'Remaining Space': 42040064,
  'Remaining Timelapse Time': 0,
  'Remaining Video Time': 4765,
  'System Busy': 0,
  'System Hot': 0},
 'Video': {'Color': 'GoPro',
  'EV Comp': '0.0',
  'Field of View': 'Wide',
  'Frames Per Second': '60',
  'ISO': '400',
  'ISO Mode': 'Max',
  'Low Light': 'ON',
  'Protune': 'ON',
  'Resolution': '2.7K',
  'Sharpness': 'Low',
  'Shutter': '1/60',
  'Video Stabilization': 'OFF',
  'White Balance': '4800K'}}